In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier, XGBRegressor
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

data = pd.read_csv("data.csv")

data_rename = data.rename(columns={"Turbidity (NTU)": "Turbidity", "DO (mg/L)": "DO_mg/L", "Temperature (C)": "Temperature_C", "Ammonium Ion": "Ammonium_Ion"})
data_finalized = data_rename.drop(["DO_mg/L", "TDS", "Ammonium_Ion", "Nitrate"], axis = 1)
data_finalized = data_finalized.drop(data_finalized.index[14:18])
data_finalized = data_finalized.drop(data_finalized.index[[1, 5, 8, 9, 13]])
data_finalized = data_finalized.reset_index(drop = True)
print(data_finalized)

                  Contaminant  Amount (mg/L)    pH  Conductivity  Turbidity  \
0                       Water            NaN  6.70          67.6       10.4   
1                        Sand          375.0  6.83          58.0       25.6   
2                        Sand          875.0  7.00          55.5       35.6   
3                        Sand         1500.0  7.18          52.0       58.2   
4                     NB Fert            5.0  6.97          70.2       23.4   
5                     NB Fert           10.0  7.08          97.5       34.5   
6  Organic Materials (Leaves)          100.0  6.67          75.0       55.0   
7  Organic Materials (Leaves)          500.0  6.57          72.0       96.0   
8  Organic Materials (Leaves)         2000.0  6.24          55.6      125.0   

   Temperature_C  
0           18.1  
1           18.3  
2           18.7  
3           19.6  
4           19.4  
5           19.2  
6           17.8  
7           17.4  
8           17.3  
